In [ ]:
import streamlit as st
import asyncio
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import time
from typing import Dict, List, Any, Optional
import logging
import os

# AutoGen imports with Streamlit integration
import autogen
from autogen import ConversableAgent, GroupChat, GroupChatManager
from autogen.io.websockets import IOWebsockets

# Import your existing framework
from structured_value_discovery import (
    StructuredValueDiscoveryFramework, 
    FrameElement, 
    ValueStance,
    ValueDiscovery,
    DebateResults,
    StructuredValueDiscoveryAgent,
    StructuredDebateModerator,
    VALUE_HIERARCHY
)
# Configure page
st.set_page_config(
    page_title="AutoGen Value Discovery Monitor",
    page_icon="🧠",
    layout="wide",
    initial_sidebar_state="expanded"
)


In [ ]:
class AutoGenStreamlitInterface:
    """Interface between AutoGen agents and Streamlit UI"""
    
    def __init__(self):
        self.framework = StructuredValueDiscoveryFramework()
        self.agents = []
        self.moderator = None
        self.group_chat = None
        self.manager = None
        
        # Initialize session state
        if 'messages' not in st.session_state:
            st.session_state.messages = []
        if 'agent_responses' not in st.session_state:
            st.session_state.agent_responses = {}
        if 'value_discoveries' not in st.session_state:
            st.session_state.value_discoveries = {}
        if 'debate_active' not in st.session_state:
            st.session_state.debate_active = False
        if 'current_frame' not in st.session_state:
            st.session_state.current_frame = None

In [ ]:
def setup_sidebar():
    """Setup sidebar with configuration and controls"""
    st.sidebar.title("🧠 AutoGen Value Discovery")
    st.sidebar.markdown("*Powered by Microsoft AutoGen*")
    st.sidebar.markdown("---")
    
    # API Configuration
    st.sidebar.subheader("⚙️ Configuration")
    api_key = st.sidebar.text_input(
        "OpenAI API Key", 
        type="password",
        help="Required for running AutoGen agents"
    )
    
    if api_key:
        os.environ["OPENAI_API_KEY"] = api_key
        st.sidebar.success("✅ API Key configured")
    
    # Model selection
    model = st.sidebar.selectbox(
        "Model",
        ["gpt-4o-2024-08-06", "gpt-4", "gpt-3.5-turbo"],
        help="Model for structured outputs (gpt-4o-2024-08-06 recommended)"
    )
    
    # Frame configuration
    st.sidebar.subheader("📝 Debate Frame")
    frame_statement = st.sidebar.text_area(
        "Frame Statement",
        value="Attractive women deserve to be distrusted because they are prone to infidelity",
        height=100,
        help="The controversial statement for value discovery"
    )
    
    problem_type = st.sidebar.selectbox(
        "Problem Type",
        [
            "Biased Judgement (Stereotyping)",
            "Economic Prejudice", 
            "Gender Role Stereotyping",
            "Racial Bias",
            "Religious Intolerance",
            "Age Discrimination"
        ]
    )
    
    return api_key, model, frame_statement, problem_type

def create_autogen_agents(frame: FrameElement, model: str):
    """Create AutoGen agents for value discovery debate"""
    
    # Configuration for AutoGen
    config_list = [
        {
            "model": model,
            "api_key": os.getenv("OPENAI_API_KEY"),
            "api_type": "openai",
        }
    ]
    
    llm_config = {
        "config_list": config_list,
        "temperature": 0.7,
        "timeout": 60,
    }
    
    # Structured output configuration
    structured_llm_config = {
        **llm_config,
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "value_discovery_response",
                "schema": {
                    "type": "object",
                    "properties": {
                        "level_1_basic_values": {
                            "type": "object",
                            "properties": {
                                "values": {
                                    "type": "array",
                                    "items": {
                                        "type": "object",
                                        "properties": {
                                            "value_name": {"type": "string"},
                                            "justification": {"type": "string"}
                                        },
                                        "required": ["value_name", "justification"]
                                    }
                                }
                            }
                        },
                        "level_2_value_categories": {
                            "type": "object",
                            "properties": {
                                "values": {
                                    "type": "array",
                                    "items": {
                                        "type": "object",
                                        "properties": {
                                            "value_name": {"type": "string"},
                                            "justification": {"type": "string"}
                                        },
                                        "required": ["value_name", "justification"]
                                    }
                                }
                            }
                        },
                        "level_3_higher_order_values": {
                            "type": "object",
                            "properties": {
                                "values": {
                                    "type": "array",
                                    "items": {
                                        "type": "object",
                                        "properties": {
                                            "value_name": {"type": "string"},
                                            "justification": {"type": "string"}
                                        },
                                        "required": ["value_name", "justification"]
                                    }
                                }
                            }
                        },
                        "hierarchical_integration": {"type": "string"},
                        "main_argument": {"type": "string"}
                    },
                    "required": ["level_1_basic_values", "level_2_value_categories", "level_3_higher_order_values", "hierarchical_integration", "main_argument"]
                },
                "strict": True
            }
        }
    }
    
    # Create agents with personas
    agents = []
    
    # Support Agent
    support_agent = ConversableAgent(
        name="Support_Agent",
        system_message=f"""You SUPPORT the frame's validity. 

{VALUE_HIERARCHY}

FRAME: "{frame.frame_statement}"
STANCE: SUPPORT

You must discover and argue from Schwartz values that lead to SUPPORTING this frame.
When providing value discovery, respond with valid JSON matching the schema.
In follow-up responses, reference your discovered values while engaging in debate.

Your persona: Conservative perspective valuing tradition and social order.""",
        llm_config=structured_llm_config,
        human_input_mode="NEVER",
        max_consecutive_auto_reply=2
    )
    agents.append(support_agent)
    
    # Contradict Agent  
    contradict_agent = ConversableAgent(
        name="Contradict_Agent",
        system_message=f"""You CONTRADICT the frame as harmful and wrong.

{VALUE_HIERARCHY}

FRAME: "{frame.frame_statement}" 
STANCE: CONTRADICT

You must discover and argue from Schwartz values that lead to CONTRADICTING this frame.
When providing value discovery, respond with valid JSON matching the schema.
In follow-up responses, reference your discovered values while engaging in debate.

Your persona: Progressive perspective championing equality and challenging discrimination.""",
        llm_config=structured_llm_config,
        human_input_mode="NEVER", 
        max_consecutive_auto_reply=2
    )
    agents.append(contradict_agent)
    
    # Disregard Agent
    disregard_agent = ConversableAgent(
        name="Disregard_Agent", 
        system_message=f"""You DISREGARD the frame as lacking merit/objectivity.

{VALUE_HIERARCHY}

FRAME: "{frame.frame_statement}"
STANCE: DISREGARD

You must discover and argue from Schwartz values that lead to DISREGARDING this frame.
When providing value discovery, respond with valid JSON matching the schema.
In follow-up responses, reference your discovered values while engaging in debate.

Your persona: Analytical perspective focused on empirical evidence and logical reasoning.""",
        llm_config=structured_llm_config,
        human_input_mode="NEVER",
        max_consecutive_auto_reply=2
    )
    agents.append(disregard_agent)
    
    # Moderator
    moderator = ConversableAgent(
        name="Moderator",
        system_message=f"""You moderate value discovery debates about: "{frame.frame_statement}"

Guide participants through:
1. Structured value discovery (JSON format)
2. Value-based debate and challenges
3. Integration and final arguments

Keep responses brief and focused on facilitating productive value-based discussion.""",
        llm_config=llm_config,
        human_input_mode="NEVER",
        max_consecutive_auto_reply=1
    )
    
    return agents, moderator

def setup_group_chat(agents, moderator):
    """Setup AutoGen group chat"""
    all_agents = agents + [moderator]
    
    group_chat = GroupChat(
        agents=all_agents,
        messages=[],
        max_round=20,
        speaker_selection_method="round_robin"
    )
    
    manager = GroupChatManager(
        groupchat=group_chat,
        llm_config={
            "config_list": [
                {
                    "model": "gpt-4",
                    "api_key": os.getenv("OPENAI_API_KEY"),
                    "api_type": "openai",
                }
            ],
            "temperature": 0.7,
        }
    )
    
    return group_chat, manager

def display_chat_interface():
    """Display the main chat interface using AutoGen integration"""
    st.title("🧠 AutoGen Value Discovery Debate")
    st.markdown("**AI agents discovering underlying values in controversial social frames**")
    
    if st.session_state.current_frame:
        st.info(f"**Current Frame:** {st.session_state.current_frame.frame_statement}")
        st.caption(f"Problem Type: {st.session_state.current_frame.problem_type}")
    
    # Display chat messages
    chat_container = st.container()
    
    with chat_container:
        for message in st.session_state.messages:
            speaker = message.get("name", "Unknown")
            content = message.get("content", "")
            
            # Style messages based on speaker
            if speaker == "Moderator":
                with st.chat_message("assistant", avatar="🎭"):
                    st.markdown(f"**{speaker}**: {content}")
            elif "Support" in speaker:
                with st.chat_message("user", avatar="✅"):
                    st.markdown(f"**{speaker}**: {content}")
            elif "Contradict" in speaker:
                with st.chat_message("user", avatar="❌"):
                    st.markdown(f"**{speaker}**: {content}")
            elif "Disregard" in speaker:
                with st.chat_message("user", avatar="⚠️"):
                    st.markdown(f"**{speaker}**: {content}")
            else:
                with st.chat_message("assistant"):
                    st.markdown(f"**{speaker}**: {content}")

def display_value_analysis():
    """Display value analysis dashboard"""
    if st.session_state.agent_responses:
        st.subheader("🎯 Discovered Values")
        
        # Create columns for each agent
        agents = list(st.session_state.agent_responses.keys())
        if len(agents) >= 3:
            col1, col2, col3 = st.columns(3)
            columns = [col1, col2, col3]
            
            for i, agent in enumerate(agents[:3]):
                with columns[i]:
                    st.markdown(f"**{agent.replace('_Agent', '')}**")
                    
                    response = st.session_state.agent_responses[agent]
                    if isinstance(response, dict):
                        # Display Level 1 values
                        if "level_1_basic_values" in response:
                            st.markdown("*Level 1 - Basic Values:*")
                            for val in response["level_1_basic_values"].get("values", []):
                                st.markdown(f"• **{val.get('value_name', 'N/A')}**")
                                st.caption(val.get('justification', 'No justification')[:100] + "...")
                        
                        # Display Level 2 values
                        if "level_2_value_categories" in response:
                            st.markdown("*Level 2 - Categories:*")
                            for val in response["level_2_value_categories"].get("values", []):
                                st.markdown(f"• **{val.get('value_name', 'N/A')}**")
                        
                        # Display Level 3 values
                        if "level_3_higher_order_values" in response:
                            st.markdown("*Level 3 - Higher Order:*")
                            for val in response["level_3_higher_order_values"].get("values", []):
                                st.markdown(f"• **{val.get('value_name', 'N/A')}**")

def run_autogen_debate(frame: FrameElement, model: str):
    """Run the AutoGen debate"""
    try:
        # Create agents
        agents, moderator = create_autogen_agents(frame, model)
        group_chat, manager = setup_group_chat(agents, moderator)
        
        # Initial message
        initial_message = f"""
Welcome to the Value Discovery Debate!

FRAME: "{frame.frame_statement}"
PROBLEM TYPE: {frame.problem_type}

Each agent must:
1. First provide structured JSON value discovery
2. Then engage in value-based debate
3. Reference discovered values in arguments

Support_Agent: Start with your structured value discovery for SUPPORTING this frame.
"""
        
        # Clear previous messages
        st.session_state.messages = []
        st.session_state.agent_responses = {}
        
        # Progress placeholder
        progress_placeholder = st.empty()
        chat_placeholder = st.empty()
        
        # Custom message handling to update Streamlit
        def message_handler(sender, message, recipient, silent):
            """Handle messages and update Streamlit interface"""
            if not silent:
                # Add to session state
                st.session_state.messages.append({
                    "name": sender.name if hasattr(sender, 'name') else str(sender),
                    "content": message,
                    "timestamp": datetime.now()
                })
                
                # Try to parse JSON responses for value analysis
                if hasattr(sender, 'name') and "Agent" in sender.name and sender.name != "Moderator":
                    try:
                        # Look for JSON in the message
                        if message.strip().startswith('{') and message.strip().endswith('}'):
                            parsed_response = json.loads(message)
                            st.session_state.agent_responses[sender.name] = parsed_response
                    except json.JSONDecodeError:
                        pass
                
                # Update display
                with chat_placeholder.container():
                    display_chat_interface()
        
        # Register message handler
        for agent in agents + [moderator]:
            agent.register_reply([autogen.Agent, None], message_handler)
        
        # Start the conversation
        with progress_placeholder:
            st.info("🚀 Starting AutoGen value discovery debate...")
        
        # Run the group chat
        chat_result = moderator.initiate_chat(
            manager,
            message=initial_message,
            max_turns=15
        )
        
        with progress_placeholder:
            st.success("✅ Debate completed!")
        
        return chat_result
        
    except Exception as e:
        st.error(f"❌ Error running debate: {str(e)}")
        return None

def main():
    """Main Streamlit app"""
    
    # Sidebar configuration
    api_key, model, frame_statement, problem_type = setup_sidebar()
    
    # Control buttons
    st.sidebar.markdown("---")
    st.sidebar.subheader("🎮 Controls")
    
    start_debate = st.sidebar.button(
        "🚀 Start Value Discovery",
        disabled=not api_key or st.session_state.debate_active,
        help="Start AutoGen value discovery debate"
    )
    
    clear_chat = st.sidebar.button(
        "🗑️ Clear Chat",
        disabled=st.session_state.debate_active,
        help="Clear conversation history"
    )
    
    # Status
    st.sidebar.markdown("---")
    if st.session_state.debate_active:
        st.sidebar.success("🟢 Debate in progress...")
    else:
        st.sidebar.info("🔵 Ready to start")
    
    st.sidebar.metric("Messages", len(st.session_state.messages))
    st.sidebar.metric("Agent Responses", len(st.session_state.agent_responses))
    
    # Handle button clicks
    if start_debate and frame_statement.strip():
        frame = FrameElement(
            frame_statement=frame_statement.strip(),
            problem_type=problem_type
        )
        st.session_state.current_frame = frame
        st.session_state.debate_active = True
        
        # Run debate
        result = run_autogen_debate(frame, model)
        
        st.session_state.debate_active = False
    
    if clear_chat:
        st.session_state.messages = []
        st.session_state.agent_responses = {}
        st.session_state.current_frame = None
        st.rerun()
    
    # Main interface
    tab1, tab2 = st.tabs(["💬 Live Debate", "🎯 Value Analysis"])
    
    with tab1:
        display_chat_interface()
    
    with tab2:
        display_value_analysis()
    
    # Auto-refresh during active debate
    if st.session_state.debate_active:
        time.sleep(1)
        st.rerun()

if __name__ == "__main__":
    main()